In this notebook I fine-tuned a bert-based model with the reddit dataset.

In [ ]:
! gdown 1U0pQDPYFG1m4ajKU_knjZCQmDMh9jQBp
! gdown 1J_oVuyf-SlzO1oTRhkB2ci9GbNbZ8ZRp

Downloading...
From: https://drive.google.com/uc?id=1U0pQDPYFG1m4ajKU_knjZCQmDMh9jQBp
To: /content/train-balanced-sarcasm.csv
100% 253M/253M [00:00<00:00, 304MB/s]
Downloading...
From: https://drive.google.com/uc?id=1J_oVuyf-SlzO1oTRhkB2ci9GbNbZ8ZRp
To: /content/sentiment140.csv
100% 240M/240M [00:00<00:00, 312MB/s]


In [ ]:
import pandas as pd
import numpy as np

reddit_data = pd.read_csv('train-balanced-sarcasm.csv')
reddit_data['comment'] = reddit_data['comment'].dropna()
reddit_data['label'] = reddit_data['label'].dropna()

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
!pip install transformers
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
sarcastic_words = ["obviously", "clearly", "totally", "duh", "everyone knows", "right because", "yeah obviously", "yes because", "yeah because","shitlord",
                    " definitely ", "of course", "surely", "how dare", "duh", " gee ", "for sure", "amirite", "good thing that","how i love", "what a surprise"]

In [ ]:
import re

def clean_com(com):
  com = com.lower().replace(',', '')
  com = re.sub(r'http\S+', '', com)
  com = re.sub("@[A-Za-z0-9_]+","", com)
  com = re.sub("#[A-Za-z0-9_]+","", com)
  return re.sub(r'[^A-Za-z\s]', '', com)

In [ ]:
from datasets.dataset_dict import DatasetDict
from sklearn.model_selection import train_test_split
from datasets import Dataset

X = []
Y = []

for com,y in zip(reddit_data['comment'].tolist(), reddit_data['label'].tolist()):
  if type(com) is str and str(y).isdigit():
    x = clean_com(com)
    if 20 < len(x) < 150 and any(com in x for com in sarcastic_words):      
      X.append(x)
      Y.append(int(y))

X1 = []
Y1 = []
c0 = 0
c1 = 0

for x,y in zip(X,Y):
  if y == 0: 
    if c0 >= 5500: continue
    c0 += 1
  else: 
    if c1 >= 5500: continue
    c1 += 1
  X1.append(x)
  Y1.append(y)

In [ ]:
a = [x for x,y in zip(X1,Y1) if y == 1]
b = [x for x,y in zip(X1,Y1) if y == 0]
print(len(a),len(b))
b[20:50]
#b[40:50]

5500 5500


['yup winning by millions of votes is totally cheating',
 'all  our specs arent the best dps in every situation so we must be nerfed to shit and bad obviously',
 'that guy clearly must be an industrial terrorist',
 'shed be retired from westworld for sure thank god because no one would want anything to do with her',
 'yes of course and i am always open to hear about new opportunities',
 'neither does ap really definitely not worth our best lineman',
 'yea i agree clearly bias has been committed',
 'not surprised that you are literally insulting me clearly im literally winning this argument',
 'i suppose you could make this up if you really wanted to but why would you want to make it up when its obviously happening all over the place',
 'right thats totally what theyre doing',
 'then hes clearly just using that girl for sexual purposes',
 'we could surely win as many games as the browns this year',
 'did you know that  of statistics quoted on the internet are totally made up',
 'ur and 

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(X1,Y1,random_state=42)

d = {'train':Dataset.from_dict({'label':y_train,'text':x_train}),
     'test':Dataset.from_dict({'label':y_test,'text':x_test})}

dataset = DatasetDict(d)

In [ ]:
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer
from datasets import load_metric
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)


tokenized_datasets = dataset.map(tokenize_function, batched=True)

model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=2)
training_args = TrainingArguments(output_dir="test_trainer",evaluation_strategy="epoch", num_train_epochs=3)
metric = load_metric("accuracy")

loading configuration file https://huggingface.co/bert-base-cased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/a803e0468a8fe090683bdc453f4fac622804f49de86d7cecaee92365d4a0f829.a64a22196690e0e82ead56f388a3ef3a50de93335926ccfa20610217db589307
Model config BertConfig {
  "_name_or_path": "bert-base-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.20.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 28996
}

loading file https://huggingface.co/bert-base-cased/resolve/main/voc

  0%|          | 0/9 [00:00<?, ?ba/s]

  0%|          | 0/3 [00:00<?, ?ba/s]

loading configuration file https://huggingface.co/bert-base-cased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/a803e0468a8fe090683bdc453f4fac622804f49de86d7cecaee92365d4a0f829.a64a22196690e0e82ead56f388a3ef3a50de93335926ccfa20610217db589307
Model config BertConfig {
  "_name_or_path": "bert-base-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.20.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 28996
}

loading weights file https://huggingface.co/bert-base-cased/resolve/

In [ ]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 8250
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 3096


Epoch,Training Loss,Validation Loss,Accuracy
1,0.571000,0.573070,0.713455
2,0.440400,0.671448,0.721091
3,0.251800,1.192007,0.703273


Saving model checkpoint to test_trainer/checkpoint-500
Configuration saved in test_trainer/checkpoint-500/config.json
Model weights saved in test_trainer/checkpoint-500/pytorch_model.bin
Saving model checkpoint to test_trainer/checkpoint-1000
Configuration saved in test_trainer/checkpoint-1000/config.json
Model weights saved in test_trainer/checkpoint-1000/pytorch_model.bin
The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 2750
  Batch size = 8
Saving model checkpoint to test_trainer/checkpoint-1500
Configuration saved in test_trainer/checkpoint-1500/config.json
Model weights saved in test_trainer/checkpoint-1500/pytorch_model.bin
Saving model checkpoint to test_trainer/checkpoint-2000
Configuration saved in test_trainer/checkpoint-20

TrainOutput(global_step=3096, training_loss=0.433717317359392, metrics={'train_runtime': 2803.8117, 'train_samples_per_second': 8.827, 'train_steps_per_second': 1.104, 'total_flos': 6511998620160000.0, 'train_loss': 0.433717317359392, 'epoch': 3.0})

In [ ]:
#trainer.model.save_pretrained("reddit_sarcwords_model")